# Exploring a Single Subject

The purpose of this notebook is to explore a single subject from the Pathstone YAC dataset.

Code snippets and functions are intended to be left alone and run as a standalone application.

Below are the common configurations that a student could reasonably expect to change during exploration. Each will contain a brief explanation as to their intended use.

### Questions

- cluster of electrodes - prof problem - send email

- area averages - prof problem; need timings; see below

- problems of rejecting bad epochs and equalizing trial counts

- expand conditions/channels for peak picking - me; trivial

- write final peaks to file, etc - me

- export waveforms to df

- average 3 points around - extend it to be optional

p3 - > (3,4)ms, (4,5), (5,6)

In [1]:
# Subject to run the picking procedure on
subject_id = '1014'

In [2]:
# EEG Configuration
filter_freqs = (1.0, 30.0) # Filter data from (low_freq, high_freq)
channel_interest = ['E75', 'E6', 'E62'] # Channels to pick for peaks
condition_interest = ['VO21', 'combine'] # Conditions to pick for peaks
epoch_tmin = -0.1 # Baseline to stimulus onset in seconds, negative implies before stimulus
epoch_tmax = 1.0 # Duration after stimulus of epoch
reject_criteria = {'eeg': 200e-6} # Reject trials that have X-microvolt peak to peak differences
p1_def   = (0.060, 0.150) # P1 peak interval definition in seconds
n170_def = (0.130, 0.220) # N170 peak interval definition in seconds
p3_def   = (0.250, 0.850) # P3 peak interval definition in seconds

# Plotting Configuration
plot_raw        = False # If true, open raw recording scrollplot
plot_components = False # If true, open component scrollplot
plot_cleaned    = False # If true, open cleaned recording scrollplot

save_epoched = False # If true, saves the epoched and cleaned version of the subject to file for further processing

In [3]:
import mne
import pandas as pd
import pylossless as ll
import matplotlib.pyplot as plt
mne.viz.set_browser_backend('qt')

project_path = '~/Documents/eeg-dev/projects/pathstone/'
raw_path = f'{project_path}/sub-YAC{subject_id}/eeg/sub-YAC{subject_id}_task-AttnCtrl_eeg.edf'
derivative_path = f'{project_path}/derivatives/pylossless/sub-YAC{subject_id}/eeg/sub-YAC{subject_id}_task-afd_eeg.edf'

ll_state = ll.LosslessPipeline()
ll_state = ll_state.load_ll_derivative(derivative_path)
ll_state.raw.info['bads'] = ll_state.flags['ch'].get_flagged()
ll_state.ica2.exclude = [index for index,comp in ll_state.flags['ic'].iterrows() if comp['ic_type'] in ['eog', 'ecg', 'muscle', 'line_noise', 'channel_noise']]
ll_state

Using qt as 2D backend.
Extracting EDF parameters from /home/tyler/Documents/eeg-dev/projects/pathstone/derivatives/pylossless/sub-YAC1014/eeg/sub-YAC1014_task-afd_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/home/tyler/Documents/eeg-dev/mne-qt-browser/mne_qt_browser/_pg_figure.py:55: FutureWarning: mne.io.pick.channel_indices_by_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
  from mne.io.pick import (_DATA_CH_TYPES_ORDER_DEFAULT,


Reading events from /home/tyler/Documents/eeg-dev/projects/pathstone/derivatives/pylossless/sub-YAC1014/eeg/sub-YAC1014_task-afd_events.tsv.
Reading channel info from /home/tyler/Documents/eeg-dev/projects/pathstone/derivatives/pylossless/sub-YAC1014/eeg/sub-YAC1014_task-afd_channels.tsv.
Reading electrode coords from /home/tyler/Documents/eeg-dev/projects/pathstone/derivatives/pylossless/sub-YAC1014/eeg/sub-YAC1014_space-CapTrak_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Reading /home/tyler/Documents/eeg-dev/projects/pathstone/derivatives/pylossless/sub-YAC1014/eeg/sub-YAC1014_task-afd_ica1_ica.fif ...
Now restoring ICA solution ...
Ready.
Reading /home/tyler/Documents/eeg-dev/projects/pathstone/derivatives/pylossless/sub-YAC1014/eeg/sub-YAC1014_task-afd_ica2_ica.fif ...
Now restoring ICA solution ...
Ready.


/home/tyler/Documents/pylossless/pylossless/pipeline.py:1199: RuntimeWarning: participants.tsv file not found for /home/tyler/Documents/eeg-dev/projects/pathstone/derivatives/pylossless/sub-YAC1014/eeg/sub-YAC1014_task-afd_eeg.edf
  self.raw = mne_bids.read_raw_bids(derivatives_path)


Raw,"('/home/tyler/Documents/eeg-dev/projects/pathstone/derivatives/pylossless/sub-YAC1014/eeg/sub-YAC1014_task-afd_eeg.edf',)"
Config,/home/tyler/Documents/eeg-dev/projects/pathstone/derivatives/pylossless/sub-YAC1014/eeg/sub-YAC1014_task-afd_ll_config.yaml
Noisy,['E25']
Bridged,['E106']
Uncorrelated,None
Rank,['E17']
EOG (Eye),"['ICA001', 'ICA004', 'ICA017', 'ICA059']"
ECG (Heart),[]
Muscle,"['ICA022', 'ICA023', 'ICA043', 'ICA050', 'ICA054']"
Line Noise,[]
Channel Noise,[]


In [ ]:
%matplotlib qt
if plot_raw:
    ll_state.raw.plot()

In [ ]:
%matplotlib qt
if plot_components:
    fig = ll_state.ica2.plot_sources(ll_state.raw, theme='light')

In [ ]:
cleaned_state = ll_state.raw.copy()
cleaned_state.load_data()
ll_state.ica2.apply(cleaned_state)
cleaned_state = cleaned_state.interpolate_bads()
cleaned_state = cleaned_state.filter(l_freq=filter_freqs[0], h_freq=filter_freqs[1])

In [ ]:
%matplotlib qt
if plot_cleaned:
    cleaned_state.plot(theme='light')

In [ ]:
raw_data = mne.io.read_raw_edf(f'sub-YAC{subject_id}/eeg/sub-YAC{subject_id}_task-AttnCtrl_eeg.edf')
base_events, event_dict = mne.events_from_annotations(raw_data)

# Programmatically relabel events
event_dict['VO24/combine'] = event_dict.pop('VO24')
event_dict['VO25/combine'] = event_dict.pop('VO25')

event_dict

In [ ]:
epochs = mne.Epochs(
    cleaned_state,
    base_events,
    picks=channel_interest,
    event_id=event_dict,
    tmin=epoch_tmin,
    tmax=epoch_tmax,
    reject=reject_criteria,
    preload=True,
    event_repeated='merge',
)
if save_epoched:
    epochs.save(f'yac_{subject_id}_pylqcr_eeg.fif', overwrite=True)
epochs

In [ ]:
%matplotlib inline

evokeds = {}
for chan in channel_interest:
    for cond in condition_interest:
        evokeds[cond] = epochs[cond].average()

    fig = mne.viz.plot_compare_evokeds(evokeds, picks=chan, combine='mean')

In [ ]:
# The below functions are helper functions for peak picking

def max_index_and_value(series, find_max=True):
    if find_max:
        index = series.idxmax()
    else:
        index = series.idxmin()
    return index, series[index]

def on_key(event, channel, condition, erp_frame): # event.key, event.x, event.y, event.xdata, event.ydata
    key_comp_order = {'1': 'p1', '2': 'n170', '3': 'p3'}
    if event.key in ['1', '2', '3']:
        closest_index = (erp_frame.index.to_series() - event.xdata).abs().idxmin()
        row_number = erp_frame.index.get_loc(closest_index)
        override_tuple =  max_index_and_value(erp_frame.iloc[range(row_number - 25, row_number + 25)], event.key in ['1', '3'])
        # erp_info[f'{condition}_{channel}_{key_comp_order[event.key]}'] = override_tuple
        erp_info[channel][condition][key_comp_order[event.key]] = override_tuple
        display(erp_info[channel][condition])

def is_figure_open(fig):
    try:
        while fig.number in plt.get_fignums():
            plt.pause(0.1)
    except:
        plt.close(fig.number)
        raise

In [ ]:
erp_info = {}

evokeds = {}
erp_frames = {}
for chan in channel_interest:
    erp_info[chan] = {}
    for cond in condition_interest:
        erp_info[chan][cond] = {}
        evokeds[cond] = epochs[cond].average()
        erp_frames[f'{cond}_{chan}'] = evokeds[cond].to_data_frame().set_index('time')[chan]
        erp_info[chan][cond]['p1'] = max_index_and_value(erp_frames[f'{cond}_{chan}'].loc[p1_def[0]:p1_def[1]])
        erp_info[chan][cond]['n170'] = max_index_and_value(erp_frames[f'{cond}_{chan}'].loc[n170_def[0]:n170_def[1]], find_max=False)
        erp_info[chan][cond]['p3'] = max_index_and_value(erp_frames[f'{cond}_{chan}'].loc[p3_def[0]:p3_def[1]])
erp_info

In [ ]:
%matplotlib qt

import time
from functools import partial

for chan in channel_interest:
    for cond in condition_interest:
        on_key_partial = partial(on_key, channel=chan, condition=cond, erp_frame=erp_frames[f'{cond}_{chan}'])
        comp_lines = [erp_info[chan][cond][comp][0] for comp in ['p1', 'n170', 'p3']]
        comp_lines.insert(0, 0) # Make sure zero is still graphed
        fig = mne.viz.plot_compare_evokeds(evokeds[cond], picks=chan, vlines=comp_lines)    
        cid = fig[0].canvas.mpl_connect('key_press_event', on_key_partial)        
        display(erp_info[chan][cond])
        is_figure_open(fig[0])

#### Break point

Confirm the values in the `erp_info` dictionary before running the final cell and writing the peaks to file.

In [ ]:
for chan in erp_info.keys():
    print('Channel: ', chan)
    for cond in erp_info[chan].keys():
        print('\tCondition: ', cond)
        for comp in erp_info[chan][cond].keys():
            print(f'\t\t{comp}\t{erp_info[chan][cond][comp][0]}\t{erp_info[chan][cond][comp][1]:.2f}')

In [ ]:
# write to file